# Programming Assignment #2: Databases

In this assignment, we will learn a little bit about databases by implementing some relational algebra operators.

## Databases

Databases store data in a collection of **tables** or **relations**, which you can think of as Excel spreadsheets. 
For example, here is how some facts about some (hypothetical) students:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W003      | Richard   | Baslim   | 2.7 |
| W001      | Maureen   | Johnson  | 4.0 |
| W012      | Lazarus   | Long     | 3.4 |
| W014      | Lummox    | Lummox   | 2.1 |
| W010      | Hazel     | Stone    | 2.9 |
| W006      | Dejah     | Thoris   | 3.4 |

> Note: What does this have to do with **relations** as discussed in class? It turns out that **relational databases** are all about relations! The first table above is a relation, in the sense that it is a subset of the cross product of StudentIds, FirstNames, LastNames, and GPAs. Those are names, but really what we mean is the crossproduct of the possible types: alphanum x alpha x alpha x rational.

An important aspect of relational databases is the notion of a **schema**, which describes the names of the tables, columns, and the types of the columns. For example, the table above may be called STUDENTS, the column names are listed in the headings, and the column types are described in the note above. There is more to schemas, but we'll ignore that in this assignment.

## TODO-0 (no points): Data Definitions and Sample Tables

Here is how we can represent that table, and some others, in ACL2. Note how the type information in the schema can be described using `defdata`. (We ignore the column names here and use column indexes instead.) Note how we define detailed schema information for each table and also give a "generic" table definition; the operators we'll be defining work on generic tables. There's nothing to do here, but be sure to execute this cell to initialize the data we'll be using later.

In [1]:
(defsnapshot from-the-top)

(defconst *STUDENTS*
    '((W003   Richard   Baslim   2.7)
      (W001   Maureen   Johnson  4.0)
      (W012   Lazarus   Long     3.4)
      (W014   Lummox    Lummox   2.1)
      (W010   Hazel     Stone    2.9)
      (W006   Dejah     Thoris   3.4)))
      
(defdata student-row
    (list symbol symbol symbol rational))
(defdata student-table
    (listof :student-row))
(check-true (student-tablep *STUDENTS*))

(defconst *COURSES*
    '((COSC2300     discrete-structures "Discrete mathematics for computer science")
      (COSC4820     databases           "Database design and implementation")
      (ENGL3666     science-fiction     "Classic works from the science fiction canon")))

(defdata courses-row
    (list :symbol :symbol :string))
(defdata courses-table
    (listof :courses-row))
(check-true (courses-tablep *COURSES*))

(defconst *TRANSCRIPT*
    '((W001   COSC2300     A)
      (W001   COSC4820     A)
      (W001   ENGL3666     A)
      (W012   COSC2300     A)
      (W012   ENGL3666     B)
      (W010   COSC2300     B)
      (W010   ENGL3666     A)))

(defdata transcript-row
    (list symbol symbol symbol))
(defdata transcript-table
    (listof transcript-row))
(check-true (transcript-tablep *TRANSCRIPT*))

(defdata generic-value (oneof :symbol :string :rational))
(defdata generic-row
    (listof :generic-value))
(defdata generic-table
    (listof :generic-row))
(check-true (generic-tablep *STUDENTS*))
(check-true (generic-tablep *COURSES*))
(check-true (generic-tablep *TRANSCRIPT*))


ACL2S !>>(DEFSNAPSHOT FROM-THE-TOP)
          20:x(DEFMACRO DEFSNAPSHOT (LABEL) ...)

Summary
Form:  ( DEFLABEL FROM-THE-TOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FROM-THE-TOP
ACL2S !>>(DEFCONST *STUDENTS*
                   '((W003 RICHARD BASLIM 27/10)
                     (W001 MAUREEN JOHNSON 4)
                     (W012 LAZARUS LONG 17/5)
                     (W014 LUMMOX LUMMOX 21/10)
                     (W010 HAZEL STONE 29/10)
                     (W006 DEJAH THORIS 17/5)))

Summary
Form:  ( DEFCONST *STUDENTS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *STUDENTS*
ACL2S !>>(DEFDATA STUDENT-ROW
                  (LIST SYMBOL SYMBOL SYMBOL RATIONAL))
 Predicate events...
Form:  ( DEFUN STUDENT-ROWP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (STUDENT-ROWP ACL2::V1) <= body -

## TODO-1 (10 pts): Projection


Let's start with **projection**. These is one of the "relational algebra" operators, and it returns a new table that has only some of the columns from the input table. For example, the result of selecting columns 1 and 3 from
the STUDENTS table above should be:

| FirstName | GPA |
|:----------|----:|
| Richard   | 2.7 |
| Maureen   | 4.0 |
| Lazarus   | 3.4 |
| Lummox    | 2.1 |
| Hazel     | 2.9 |
| Dejah     | 3.4 |

Note that we are using 0-based indexing here, as is traditional in computer science.

Your first task is to write the function `(projection indexes relation)` that performs this operation. For example, the projection above corresponds to `(projection '(1 3) *STUDENTS*)`.

> Hint: You may find it helpful to write **two** functions, the `projection` function and a helper function that projects a single row.

> Hint: Play around with the built-in function `nth`, as in `(nth 3 '(1 2 3 4 5))`. It will help you to write this function.

In [88]:
(defsnapshot todo-1)




(check-expect (projection 1 3 *STUDENTS*)'((Richard 2.7
                                            Maureen 4.0
                                            Lazarus 3.4
                                            Lummox 2.1
                                            Haze l2.9
                                            Dejah 3.4)))

(check-expect (projection 0 3 *STUDENTS*)'((W003 2.7)
                                          (W001 4.0)
                                          (W012 3.4)
                                          (W014 2.1)
                                          (W010 2.9)
                                          (W006 3.4)))

(check-expect (projection 1 2 *STUDENTS*)'((Richard   Baslim)
                                          (Maureen   Johnson)
                                          (Lazarus   Long)
                                          (Lummox    Lummox)
                                          (Hazel     Stone)
                                          (Dejah     Thoris)))

(check-expect (projection 2 3 *STUDENTS*)'((Baslim   2.7)
                                          (Johnson  4.0)
                                          (Long     3.4)
                                          (Lummox   2.1)
                                          (Stone    2.9)
                                          (Thoris   3.4)))

(check-expect (projection 1 3 *STUDENTS*)'((W003   Richard)
                                          (W001   Maureen)
                                          (W012   Lazarus)
                                          (W014   Lummox)
                                          (W010   Hazel)
                                          (W006   Dejah)))







ACL2S !>>(DEFSNAPSHOT TODO-1)
          33:x(DEFDATA GENERIC-TABLE (LISTOF GENERIC-ROW))

Summary
Form:  ( DEFLABEL TODO-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-1
ACL2S !>>(DEFDATA COORD
                  (LIST (RANGE INTEGER (0 <= _ <= 3))
                        (RANGE INTEGER (0 <= _ <= 3))))
 Predicate events...
Form:  ( DEFUN COORDP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (COORDP ACL2::V1) <= body -- not complete. 
Reasons: 
("Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

 (COORDP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules."
 "Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

Form:  ( DEFTHM COORD=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-C

## TODO-2 (10 pts): Selection, Part 1

If projection lets you keep some of the columns of a table, **selection** is all about keeping some rows of the table and discarding the rest. We'll start with the simplest version of selection, and that is to select the rows that have a given value in one of their columns, e.g., the rows with gpa (i.e., column #3) equal to 3.4:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W012      | Lazarus   | Long     | 3.4 |
| W006      | Dejah     | Thoris   | 3.4 |

Implement the function `(selection-constant index value relation)` that selects the rows in relation where the column given by the index is equal to the value provided. E.g., the call `(selection-constant 3 3.4 *STUDENTS*)` generates the results above.

> Hint: Remember that you can always define auxiliary functions. Also, the hints to TODO-1 apply here as well.

> Hint: Use the `generic-` types defined in TODO-0.

In [ ]:
(defsnapshot todo-2)






















## TODO-3 (10 pts): Selection, Part 2

Next, let's look at a different type of selection. Here, we will select the rows where one column has the same value as another column. For example, we can select the rows where the FirstName and LastName have the same value:

| StudentId | FirstName | LastName | GPA |
|:----------|:----------|:---------|----:|
| W014      | Lummox    | Lummox   | 2.1 |

Implement the function `(selection-column index1 index2 relation)` that selects the rows in relation where the two columns have the same value. E.g., the call `(selection-column 1 2 *STUDENTS*)` generates the results above.

> Note: You may not think this particular operation is useful, but just wait a couple of TODOs. It turns out to be part of something very important in databases, called the **join** operator.

In [ ]:
(defsnapshot todo-3)





















## TODO-4 (10 pts): Using Relational Algebra

Using the **functions you already defined**, write a `check-expect` expression that finds the first name and last name (only) of anybody who has a GPA of exactly 3.4. This should give you an idea of how relational algebra works in practice. In fact, here is one version of the expression I'm asking you to implement, but in relational algebra  syntax:

$$\pi_{FirstName, LastName}(\sigma_{GPA=3.4}(STUDENTS))$$

> Curiosity: The Greek letters $\pi$ and $\sigma$ stand for **p**rojection and **s**election, respectively.

> Note: This question is just meant to give you an idea of how relational algebra is used to **query** databases, so you can get information from the facts stored in the database.

In [ ]:
(defsnapshot todo-4)
















## TODO-5 (10 pts): Cross Product

Now let's implement another important operation, and the first that involves more than one table. The cross-product is exactly the same as defined in class. The cross-product of table `A` with `m` columns and `B` with `n` columns is a table with `m+n` columns where the first `m` columns come from `A` and the last `n` columns come from `B`. For example, the cross product of STUDENTS and COURSES is

| StudentId | FirstName | LastName | GPA | CourseId   | Title               | Description                                |
|:----------|:----------|:---------|----:|:-----------|:--------------------|:-------------------------------------------------|
| W003      | Richard   | Baslim   | 2.7 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W003      | Richard   | Baslim   | 2.7 | COSC4820   | databases           | "Database design and implementation"           |
| W003      | Richard   | Baslim   | 2.7 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W001      | Maureen   | Johnson  | 4.0 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W001      | Maureen   | Johnson  | 4.0 | COSC4820   | databases           | "Database design and implementation"           |
| W001      | Maureen   | Johnson  | 4.0 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W012      | Lazarus   | Long     | 3.4 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W012      | Lazarus   | Long     | 3.4 | COSC4820   | databases           | "Database design and implementation"           |
| W012      | Lazarus   | Long     | 3.4 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W014      | Lummox    | Lummox   | 2.1 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W014      | Lummox    | Lummox   | 2.1 | COSC4820   | databases           | "Database design and implementation"           |
| W014      | Lummox    | Lummox   | 2.1 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W010      | Hazel     | Stone    | 2.9 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W010      | Hazel     | Stone    | 2.9 | COSC4820   | databases           | "Database design and implementation"           |
| W010      | Hazel     | Stone    | 2.9 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |
| W006      | Dejah     | Thoris   | 3.4 | COSC2300   | discrete-structures | "Discrete mathematics for computer science"    |
| W006      | Dejah     | Thoris   | 3.4 | COSC4820   | databases           | "Database design and implementation"           |
| W006      | Dejah     | Thoris   | 3.4 | ENGL3666   | science-fiction     | "Classic works from the science fiction canon" |

Write the function `(cross relation1 relation2)` that returns the cross-product of two relations, e.g., `(cross *STUDENTS* *COURSES*)`.

In [ ]:
(defsnapshot todo-5)




























## TODO-6 (10 pts): Join

Finally, we can implement **join**, the most important relational algebra operation of all. In fact, much research into databases has gone into finding ways to implement **join** efficiently.

A join is nothing more than a cross product followed by a selection, and the selection says that one column of the first relation is equal to some column in the second relation. For example, we can join STUDENTS and TRANSCRIPT on the first column of both tables, to get

| StudentId | FirstName | LastName | GPA | StudentId | CourseId   | Grade |
|:----------|:----------|:---------|----:|:----------|:-----------|:------|
| W001      | Maureen   | Johnson  | 4.0 | W001      | COSC2300   | A     |
| W001      | Maureen   | Johnson  | 4.0 | W001      | COSC4820   | A     |
| W001      | Maureen   | Johnson  | 4.0 | W001      | ENGL3666   | A     |
| W012      | Lazarus   | Long     | 3.4 | W012      | COSC2300   | A     |
| W012      | Lazarus   | Long     | 3.4 | W012      | ENGL3666   | B     |
| W010      | Hazel     | Stone    | 2.9 | W010      | COSC2300   | B     |
| W010      | Hazel     | Stone    | 2.9 | W010      | ENGL3666   | A     |

In relational algebra notation, we would write this as $STUDENTS \bowtie_{StudentId,StudentId} TRANSCRIPT$.
Notice that this is starting to give us very useful information! We have students (with names) and the courses they've taken. If you do another join, you can get the student names **and** the course names. (See TODO-7.)

Using the functions you've already defined, implement the function `(join index1 index2 relation1 relation2)` that joins the two relations on the columns specified. E.g., the table above is the result of `(join 0 0 *STUDENTS* *TRANSCRIPT*)`.


In [ ]:
(defsnapshot todo-6)



















## TODO-7 (10 pts): Another Sample Query

Similar to TODO04, this is another exercise in writing relational algebra expressions to query a database. Using the functions already defined, use `check-expect` to find the course ID, course title, and grade of all the courses that the student with first name Lazarus and last name Long has taken. Again, here is a way of doing this using relational algebra syntax:

$$\pi_{CourseId, Title, Grade}(\sigma_{FirstName,Lazarus}(\sigma_{LastName,Long}((STUDENTS \bowtie_{StudentId,StrudentId} (TRANSCRIPT \bowtie_{CourseId,CourseId} COURSES)))))$$

In [ ]:
(defsnapshot todo-7)

















## TODO-8 (10 pts): Pushing Selections into Joins

A key part of the answer to TODO-6 looked like this:

$$\sigma_{LastName, Long}(STUDENTS \bowtie_{StudentId, StudentId} TRANSCRIPT)$$

This computes the join of the two tables and then selects only the relevant rows. But what if we did it the other way around? First, select the relevant rows from the first table, then join those rows with the second table. You should convince yourself that that will return the same answer.

Now, let's use ACL2 to guarantee that this is the case. Use `test?` to verify that these two execution ideas will always return the same values:
* $\sigma_{i,x}(A \bowtie_{j,k} B)$
* ($\sigma_{i,x}A) \bowtie_{j,k} B$

> Note: In reality, of course, the two expressions will return the same values, but one of them may be much faster than the other. Which is better and how much better will depend on details about the amount of data and the way the data is organized on disk. A good relational database (think Oracle, MySQL, PostgreSQL, SQL Server, etc.) will consider many different but equivalent *query execution plans* and pick the one that they think will result in the best overall performance. This happens essentially transparently to you, and you will only have to interfere when the query execution is just too slow. When that happens, you can see the query plan that the database came up with (i.e., the relational algebra expression it chose as "the best one") and modify the database accordingly. Details are in a future course, but way beyond this one.

Hint: Take some time to think through the hypothesis. E.g., does this theorem really hold for all values of $i$, $j$, and $k$? Do you need some extra assumptions about $A$ or $B$?

Hint: No matter what, remember that all variables need to be typed explicitly in the hypotheses.

In [ ]:
(defsnapshot todo-8)





















## EC-1 (20 pts): Prove It

Use `thm` to prove that the property from TODO-8 holds. Warning: This is hard.

In [ ]:
(defsnapshot ec-1)














